In [1]:
import ee
ee.Authenticate()
ee.Initialize()


Successfully saved authorization token.


In [2]:
# Import datasets
gaul = ee.FeatureCollection("FAO/GAUL/2015/level2")
noaa_ndvi = ee.ImageCollection("NOAA/CDR/AVHRR/NDVI/V5")
modis_ndvi = ee.ImageCollection("MODIS/061/MOD13Q1")
google_ndvi = ee.ImageCollection("MODIS/MOD09GA_006_NDVI")

In [34]:
REGION_NAMES = ['El Jadida', 'Gabon', 'Isiolo', 'Khouribga', 'Kitui', 'Laikipia', 'Marsabit', 'Maynas', 'Para', 'Safi', 'Selangor', 'Sri Lanka']
REGION_TYPES = [2, 0, 2, 2, 2, 2, 2, 2, 1, 2, 1, 0]
SCALE = 250 # m
MAX_PIXELS = 100000000

In [15]:
def process_google_ndvi_image(image, county, scale, max_pixels):
    date = image.get('system:time_start')
    region_mask =  ee.Image.constant(1).clip(county)
    county_bounds = county.geometry().bounds()
    mask = region_mask
    masked_image = image.updateMask(mask)
    value = masked_image.select('NDVI').reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=county_bounds,
        scale=scale,
        maxPixels=max_pixels).get('NDVI')
    feature = ee.Feature(None, {'date': ee.Date(date).format('MMM dd, YYYY'),
                                'ndvi': value})
    return feature

In [35]:
task_list = []
for region, region_type in zip(REGION_NAMES, REGION_TYPES):
    region_printable = region.lower().replace(' ', '')
    county = gaul.filter(ee.Filter.eq(f'ADM{region_type}_NAME', region))
    google_ndvi_data = google_ndvi.map(lambda img: process_google_ndvi_image(img, 
                                                                          county, 
                                                                          SCALE, 
                                                                          MAX_PIXELS))
    task = ee.batch.Export.table.toDrive(google_ndvi_data, f'google_ndvi_{region_printable}', region_printable, selectors = 'date, ndvi')
    task.start()
    task_list.append(task)

In [49]:
for task in task_list:
    status = task.status()
    description = status['description']
    state = status['state']
    if state != 'COMPLETED':
        print(state, description)